In [52]:
import os
import json
import math
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


# Création d'un masque à partir du GeoJson et d'une liste de features

- Cette version traite une image à la fois. 
**V2 possible: prendre en entrée un fichier texte contenant les noms des cartes à traiter**
- On considère une liste donnée de feattypes donnée plus bas
- Le masque en sortie est un png de la même taille que le GeoTIFF en entrée, codé sur un canal 8 bits
- Si un pixel appartient à au moins deux objets, il prend la valeur de pixel somme 
**V2 possible: donner une règle de priorité entre les types d'objet: qui passe devant qui ? ex runway marking devant runwayelement.**


In [53]:
# A MODIFIER AVEC LES BONS CHEMINS POUR RUN LE CODE SUR VOTRE MACHINE

path_GeoTIFF = "/home/limbothai/PIE/Airport_Feature_Auto_Detection/GeoTIFF/"
path_GeoJson = "/home/limbothai/PIE/Airport_Feature_Auto_Detection/GeoJson/"

#image considérée
name_map = "LIRP"

#chemin de sortie pour le masque png
outfolder = "/home/limbothai/PIE/Airport_Feature_Auto_Detection/Masks"  
os.makedirs(outfolder, exist_ok=True)

In [54]:
# Liste des features d'intérêt et color mapping (1 canal nuances de gris de 1 à 255, 0 est background)
color_mapping = {
    'Apron.apronelement': 10,
    'Runway.runwayelement': 50,
    'ServiceRoad.serviceroad': 100,
}
features_list = ['Apron.apronelement','Runway.runwayelement','ServiceRoad.serviceroad']

In [55]:
# Fonctions de création de masques par lecture du GeoJson
# TO DO: V2 qui gère la superposition d'objets. Idée: un petit objet inclut dans un autre doit avoir priorité sur l'affichage 

def poly_from_utm(polygon, transform):
    poly_pts = []
    
    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        
        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))
        
    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

# creating binary mask for field/not_filed segmentation.
def create_mask(features_list,src,df):
    im_size = (src.meta['height'], src.meta['width'])
    whole_mask = np.zeros([src.meta['height'],src.meta['width']])

    for i,f in enumerate(features_list):
        poly_shp = []
        data_geom = df[df['feattype'] == {'value': f}].geometry
        for g in data_geom.values:

            poly = poly_from_utm(g, src.meta['transform'])
            poly_shp.append(poly)

            if g.geom_type == 'Polygon':
                poly = poly_from_utm(g, src.meta['transform'])
                poly_shp.append(poly)
            else:
                for p in g:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)
        if poly_shp:
            mask = rasterize(shapes=poly_shp,out_shape=im_size)*color_mapping[f]
            whole_mask = whole_mask + mask
    
    return whole_mask


In [56]:
img_name = name_map+"_ortho.tif"
directory = os.fsencode(path_GeoTIFF)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename==img_name:
        with rasterio.open(path_GeoTIFF+"{}".format(filename)) as src:
            img = src.read()
            meta = src.meta
            filepath = path_GeoJson+"product_ADBLucem_"+name_map+".json"
            df = polygon_df(filepath)
            
            whole_mask = create_mask(features_list,src,df)
            
            # Just stores the different pixel values in the mask array, and print them.
            a=set([])
            for row in whole_mask:
                for elt in row:
                    a.add(elt)
            print("Pixel values: "+str(a))
            
            # Just prints warning if two objects are superimposed
            a.remove(0)
            for elt in a:
                boolean=False
                for key in color_mapping:
                    if(color_mapping[key] == elt):
                        boolean=True;
                if not boolean:
                    print("A superposition was detected !")
                    
            
            print(whole_mask.shape)
            
#             #Rescale to 0-255 and convert to uint8
#             rescaled = (255.0 / data.max() * (data - data.min())).astype(np.uint8)

            # Save numpy array into png mask 1 channel
            cv2.imwrite(outfolder + '/masktest_{}.png'.format(img_name.split('_')[0]),whole_mask)


Pixel values: {0.0, 50.0}
(6200, 7400)


# OLD: exploration et manipulation d'un GeoJson

### Open image with rasterio:

In [1]:
with rasterio.open(path_GeoTIFF+"/"+name_map+"_ortho.tif") as src:
    img = src.read()
    meta = src.meta

img = reshape_as_image(img)



NameError: name 'rasterio' is not defined

### Read Geojson as  a dataframe:

In [3]:
def polygon_df(filepath):
    df = gpd.read_file(filepath)
    df = df.drop(columns=['originated', 'readonly',
           'notvalidated', 'lock', 'elev', 'hacc', 'iata', 'name', 'idarpt',
           'acft', 'idnumber', 'termref', 'pcn', 'restacft', 'status', 'surftype',
           'length', 'width', 'color', 'direc', 'style', 'rwymktyp', 'asda',
           'availPavedSurfFromThr', 'brngmag', 'brngtrue', 'cat', 'ellipse',
           'geound', 'lda', 'rops_landing_length', 'rwyslope', 'tdze', 'tdzslope',
           'thrtype', 'toda', 'tora', 'vasis', 'bridge', 'gsurftyp', 'runwayexit',
           'imagery_date'])
    
    return df[df["geometry"].geom_type == 'Polygon']

#### Only polygon:

In [20]:
df = polygon_df(path_GeoJson+"/product_ADBLucem_"+name_map+".json")
df.head()

,id,feattype,label_id,name_id,aprontyp,docking,fuel,gndpower,idapron,jetway,...,featbase,wingspan,maxspeed,catstop,idlin,rwyahtxt,height,material,plysttyp,geometry
1,ALLADB|ADBLucem|apronelement|84115,{'value': 'Apron.apronelement'},{'value': '$UNK'},{'value': '$UNK'},{'value': None},None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((2.87494 42.73283, 2.87477 42.73266, ..."
2,ALLADB|ADBLucem|apronelement|84116,{'value': 'Apron.apronelement'},{'value': '$UNK'},{'value': '$UNK'},{'value': None},None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((2.87698 42.73543, 2.87700 42.73494, ..."
3,ALLADB|ADBLucem|apronelement|84117,{'value': 'Apron.apronelement'},{'value': 'EAS APRON'},{'value': 'EAS APRON'},{'value': None},None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((2.86649 42.74389, 2.86732 42.74386, ..."
4,ALLADB|ADBLucem|apronelement|84118,{'value': 'Apron.apronelement'},{'value': '$UNK'},{'value': '$UNK'},{'value': None},None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((2.87719 42.73294, 2.87719 42.73291, ..."
5,ALLADB|ADBLucem|apronelement|84119,{'value': 'Apron.apronelement'},{'value': '$UNK'},{'value': '$UNK'},{'value': None},None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"POLYGON ((2.87736 42.73311, 2.87799 42.73276, ..."


### Liste des features contenues dans le GeoJSON

In [21]:
features_list = []
for e in df['feattype']:
    if e['value'] not in features_list:
        features_list.append(e['value'])

print(features_list)

['Apron.apronelement', 'FinalApproachAndTakeOffArea.finalapproachandtakeoffarea', 'ParkingStandArea.parkingstandarea', 'Runway.runwaydisplacedarea', 'Runway.runwayelement', 'Runway.runwaymarking', 'Runway.runwayshoulder', 'ServiceRoad.serviceroad', 'Stopway.stopway', '0', 'Taxiway.taxiwayshoulder', 'TouchDownLiftOffArea.touchdownliftoffarea', 'VerticalStructure.verticalpolygonalstructure', 'Water.water']
